---

# CSCI 3202, Fall 2023
# Mancala Project - Outline


In [91]:
import copy, math, time, random
from random import randint
random.seed(109)

1. Play 100 games of random player against random player
    - What percentage of games does each player (1st or 2nd) win?
    - On average, how many moves does it take to win?
2. Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
3. Play 100 games with the random player against the minimax AI player at a depth of 5 plies
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Is your AI player better than random chance? Write a paragraph or two describing why or why not.
4. Play 100 games with the random player against the Alpha-Beta AI player at a depth of 5 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Are your results for this part different from those for your minimax AI player? Write a paragraph or two describing why or why not.
5. (Extra Credit, 10 points). Play 100 games with the random player against the
    - Alpha-Beta AI player at a depth of 10 plies
    - How long does it take for a single game to run to completion?
    - What percentage of games does each player (AI or random) win?
    - On average, how many moves does it take to win?
    - Does increasing the number of plies improve the play for our AI player? Why or why not?

explanation for deepcopy - copy library
https://www.scaler.com/topics/copy-in-python/

In [100]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        self.stones_per_pit = stones_per_pit
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board) - 1
    
        self.num_plays = 0
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
        self.p1_wins = 0
        self.p2_wins = 0
        self.ties = 0
        

    def display_board(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                       self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i], player_2_pits[-(i + 1)],
                                                      self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
    
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        self.winning_eval()
        
        valid_moves = []
        if self.current_player == 1:
            for pit_index in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                if self.board[pit_index] > 0:
                    valid_moves.append(pit_index+1)
        else:
            for pit_index in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                if self.board[pit_index] > 0:
                    valid_moves.append(pit_index - self.pits_per_player)
                    
        if valid_moves:
            random_move = random.choice(valid_moves)
            return random_move
        else:
            self.winning_eval()
            return 1
        
    def generate_board(self):
        self.board = [self.stones_per_pit] * ((self.pits_per_player+1) * 2)
        self.current_player = 1
        self.moves = []
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        To do this, we check if pit is not empty
        Then we make sure it is within [1 -> len(pitsperplayer)]
        """
        ##Check if out of bounds first
        if(pit < 0 or pit >= len(self.board)):
            #print("pit out of bounds")
            return(False)
        
        ##Check if pit is empty
        if (self.board[pit] == 0):
            #print("pit is empty")
            return(False)
        else:
            ##Make sure only doing available pits
            if(self.current_player == 1):
                #print(self.p1_pits_index[0], " <= ", pit, " <= ", self.p1_pits_index[1])
                return (pit >= self.p1_pits_index[0] and pit <= self.p1_pits_index[1])
            if(self.current_player == 2):
                #print(self.p2_pits_index[0], " <= ", pit, " <= ", self.p2_pits_index[1])
                return (pit >= self.p2_pits_index[0] and pit <= self.p2_pits_index[1])

    def play(self, pit, cur_player, debug = False):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        self.current_player = cur_player
        ##Convert pit into actual pit index
        pit = pit - 1
        if self.current_player == 2:
            pit = pit + len(self.board)//2
        
        # write your code here
        ##Check Valid Move
        if(self.valid_move(pit) == False):
            print(pit)
            print("INVALID MOVE")
            return self.board
        ##Check if game is over
        if(self.winning_eval()):
            print("GAME_OVER")
            return self.board
        
        ##Take pieces and distribute
        counter = self.board[pit]
        self.board[pit] = 0
        while(counter > 0):
            ##make sure no out of bounds
            pit = (pit + 1)%len(self.board)
            
            ##make sure you dont put beads in other persons mancala
            if (self.current_player == 1 and pit == self.p2_mancala_index):
                ##print("skipping the mancala for opposite player")
                ##print(pit, self.p2_mancala_index)
                continue
            if (self.current_player == 2 and pit == self.p1_mancala_index):
                ##print("skipping the mancala for opposite player")
                ##print(pit, self.p1_mancala_index)
                continue
                
        ##check if last bead rule
            if (counter == 1 and self.board[pit] == 0 and pit != self.p1_mancala_index and self.current_player == 1 and pit >= self.p1_pits_index[0] and pit <= self.p1_pits_index[1]):
                    opposite_pit = self.p1_mancala_index + (self.p1_mancala_index - pit)
                    self.board[self.p1_mancala_index] += self.board[opposite_pit] + 1
                    self.board[opposite_pit] = 0
                    counter -= 1
                    continue
            if (counter == 1 and self.board[pit] == 0 and pit != self.p2_mancala_index and self.current_player == 2 and pit >= self.p2_pits_index[0] and pit <= self.p2_pits_index[1]):
                    opposite_pit = self.p1_mancala_index - (pit-self.p1_mancala_index)
                    self.board[self.p2_mancala_index] += self.board[opposite_pit] + 1
                    self.board[opposite_pit] = 0
                    counter -= 1
                    continue
                    
                    
            self.board[pit] += 1
            counter -= 1
        
        ##Change turns
        if self.current_player == 1:
            self.current_player = 2
        else:
            self.current_player = 1
        
        return self.board
        
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        i = self.p1_pits_index[0]
        p1DONE = True
        p2DONE = True
        while(i <= self.p1_pits_index[1]):
            if self.board[i] != 0:
                p1DONE = False
            i+= 1
        j = self.p2_pits_index[0]
        while(j <= self.p2_pits_index[1]):
            if self.board[j] != 0:
                p2DONE = False
            j+=1
            
        if p1DONE or p2DONE:
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                self.p1_wins += 1
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                self.p2_wins += 1
            else:
                self.ties += 1
        return(p1DONE or p2DONE)
    
    def match_analysis(self, games = 100):
        for i in range(games):
            self.generate_board()
            while self.winning_eval() == False:
                self.play(self.random_move_generator(),1)
                if(self.winning_eval()):
                    break
                self.play(self.random_move_generator(),2)

In [94]:
class MancalaAI:
    def __init__(self, depth, state):
        self.depth = depth
        self.state = state
    
    def minimax(self, state, depth, maximizing_player, cur_player):
        if maximizing_player:
            # generate all possible states for the maximizing player, and recurse 
            # until you reach the stop condition - terminal state
            for i in possible_valid_moves:
                state.play()
                minimax(state, depth-1, False, 3 - cur_player) # minimizing players' move
        else:
            # generate all possible states for the maximizing player, and recurse
            for i in possible_valid_moves:
                state.play()
                minimax(state, depth-1, True, 3 - cur_player) # maximizing players' move
        pass
    
     def minimax_alpha_beta(self, state, depth, alpha, beta, maximizing_player, cur_player):
        pass

    def best_move(self, state, alpha_beta = False):
        
        for i in possible_valid_moves:
            if state.current_player == 1:
                value = minimax(state, depth, True, state.current_player)
                # find the max value out of all options, and return the move corresponding to that max value
            else:
                if state.current_player == 2:
                value = minimax(state, depth, False, state.current_player)
                # find the min value out of all options, and return the move corresponding to that min value
        return best_move
        pass

    def evaluate_state(self, state):
        # Utility function  :- Difference between P1 mancala and p2 mancala
        pass

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

## Test 100 Random vs Random Games

In [113]:
newGame = Mancala()
newGame.match_analysis()
print("Games Finished")
print(newGame.p1_wins)
print(newGame.p2_wins)
print(newGame.ties)

Games Finished
52
45
3
